In [2]:
# Initial Load - let's just do 2022 data. 2020 might be skewed, covid.

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import matplotlib.pyplot as plt
import seaborn as sns

spark = SparkSession.builder.appName("FlightDataAnalysis").config("spark.driver.memory", "4g").getOrCreate()

# Load your flight data into a Spark DataFrame
data_path = "gs://msca-bdp-student-gcs/Group4_Final_Project/archive/Combined_Flights_2022.csv"
data = spark.read.csv(data_path, header=True, inferSchema=True)

# Data preprocessing (replace this with your specific cleaning logic)
# Example: Remove rows with missing values
data_cleaned = data.dropna()

# Summary statistics for numeric variables
numeric_vars = ['DepDelayMinutes', 'ArrDelayMinutes']
summary_numeric = data_cleaned.select(numeric_vars).describe()

# Summary statistics for categorical variables
categorical_vars = ['Airline', 'Origin', 'Dest', 'DepartureDelayGroups', 'ArrivalDelayGroups', 'Month', 'DayofMonth', 'DayOfWeek', 'Marketing_Airline_Network']
summary_categorical = data_cleaned.select(categorical_vars).describe()




In [13]:
# Output and print the summary statistics
print("Summary statistics for numeric variables:")
summary_numeric.show()

print("Summary statistics for categorical variables:")
summary_categorical.show()

Summary statistics for numeric variables:
+-------+------------------+------------------+
|summary|   DepDelayMinutes|   ArrDelayMinutes|
+-------+------------------+------------------+
|  count|           3944916|           3944916|
|   mean|15.940134086505264|15.783071426615928|
| stddev| 52.16239928825909| 51.98423581313936|
|    min|               0.0|               0.0|
|    max|            7223.0|            7232.0|
+-------+------------------+------------------+

Summary statistics for categorical variables:
+-------+--------------------+-------+-------+--------------------+-------------------+------------------+-----------------+------------------+-------------------------+
|summary|             Airline| Origin|   Dest|DepartureDelayGroups| ArrivalDelayGroups|             Month|       DayofMonth|         DayOfWeek|Marketing_Airline_Network|
+-------+--------------------+-------+-------+--------------------+-------------------+------------------+-----------------+---------------

In [12]:
# Perform time series analysis on departure delays
data_cleaned = data_cleaned.withColumn("FlightDate", F.to_date("FlightDate", "yyyy-MM-dd"))
data_cleaned = data_cleaned.withColumn("Year", F.year("FlightDate"))

# Group the data by year and calculate the average departure and arrival delays
time_series_dep_delay = data_cleaned.groupBy("Year").agg(F.avg("DepDelayMinutes").alias("AvgDepDelayMinutes"))
time_series_arr_delay = data_cleaned.groupBy("Year").agg(F.avg("ArrDelayMinutes").alias("AvgArrDelayMinutes"))

# Convert the result to Pandas DataFrame for plotting
time_series_dep_delay_df = time_series_dep_delay.toPandas()
time_series_arr_delay_df = time_series_arr_delay.toPandas()

# Plot the time series of departure and arrival delays
plt.figure(figsize=(12, 6))
plt.plot(time_series_dep_delay_df["Year"], time_series_dep_delay_df["AvgDepDelayMinutes"], label="Average Departure Delay")
plt.plot(time_series_arr_delay_df["Year"], time_series_arr_delay_df["AvgArrDelayMinutes"], label="Average Arrival Delay")
plt.xlabel("Year")
plt.ylabel("Average Delay (Minutes)")
plt.title("Time Series of Departure and Arrival Delays Over Years")
plt.legend()
plt.show()

# Stop the SparkSession when you're done
spark.stop()



Py4JJavaError: An error occurred while calling o227.collectToPython.
: org.apache.spark.sql.catalyst.errors.package$TreeNodeException: execute, tree:
Exchange hashpartitioning(Year#4272, 200), ENSURE_REQUIREMENTS, [id=#352]
+- *(1) HashAggregate(keys=[Year#4272], functions=[partial_avg(DepDelayMinutes#2478)], output=[Year#4272, sum#4466, count#4467L])
   +- *(1) Project [DepDelayMinutes#2478, year(cast(gettimestamp(cast(gettimestamp(FlightDate#2470, yyyy-MM-dd, Some(Etc/UTC), false) as date), yyyy-MM-dd, Some(Etc/UTC), false) as date)) AS Year#4272]
      +- *(1) Filter AtLeastNNulls(n, FlightDate#2470,Airline#2471,Origin#2472,Dest#2473,Cancelled#2474,Diverted#2475,CRSDepTime#2476,DepTime#2477,DepDelayMinutes#2478,DepDelay#2479,ArrTime#2480,ArrDelayMinutes#2481,AirTime#2482,CRSElapsedTime#2483,ActualElapsedTime#2484,Distance#2485,Year#2486,Quarter#2487,Month#2488,DayofMonth#2489,DayOfWeek#2490,Marketing_Airline_Network#2491,Operated_or_Branded_Code_Share_Partners#2492,DOT_ID_Marketing_

FlightDate: The date of the flight, which can be used to analyze trends and patterns over time.

Airline: The airline operating the flight, as different airlines may have varying on-time performance records.

Origin and Dest: The departure and arrival airports, which can be used to examine how on-time performance varies by location.

CRSDepTime and CRSArrTime: The scheduled departure and arrival times, which are important for calculating on-time performance.

DepTime and ArrTime: The actual departure and arrival times, which are needed to calculate the actual on-time performance.

DepDelayMinutes and ArrDelayMinutes: The delay in minutes for departure and arrival, which directly relate to on-time performance.

DepDel15 and ArrDel15: Binary indicators of whether a flight was delayed by 15 minutes or more. These can be used to categorize flights as on-time or delayed.

DepartureDelayGroups and ArrivalDelayGroups: Categorical groups that classify flights based on their departure and arrival delays.

Month, DayofMonth, and DayOfWeek: Date-related variables that allow you to analyze how on-time performance varies by month, day of the month, and day of the week.

Marketing_Airline_Network: Information about the airline's marketing network can provide insights into the airline's marketing strategies and how they affect on-time performance.



**Most rich: Marketing_airline_network -- can do k-means clustering?
DepDelayMinutes, ArrDelayMinutes, Origin, Destination, Month, Day of month, Day of week.**

Some very rough plots:

In [ ]:
A lot of different approaches. We Can do
1/ seasonality? ACF/PACF, according to time? lets try that -- 